Full name: Nguyễn Thế Hoàng

Student ID: 20120090

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# HW2: Parallel Execution in CUDA

**To compile your file, you can use this command:** \
`nvcc tên-file.cu -o tên-file-chạy` \
***You can use Vietnamese to anwser the questions***

In [ ]:
from numba import cuda

major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

wkDir = './'
p1Path = f'{wkDir}HW2_P1.cu'
p1Exe = f'{wkDir}P1.exe'

In [ ]:
!nvcc -arch=sm_{major}{minor} --ptxas-options=-v {p1Path} -o {p1Exe}
!nvprof {p1Exe}

## Question 1A

| Kernel | Execute Time (ms) |
| -------- | -------- |
| Warmup | 0.230 |
| Host | 44.758 |
| Kernel 1 | 2.710 |
| Kernel 2 | 1.813 |
| Kernel 3 | 1.308 |

### Reasoning

- In the Kernel 1: this segment of condition checking

    ```if (threadIdx.x % stride == 0)```

    only let the thread at the head of the current stride do computation. So this will cause warp divergence. However, the unrolling in this kernel also help speed things a little (each thread block sums two consecutive data block).

- In the Kernel 2: the thread is not fixed in the elements it need to compute. The thread can be repositioned in each stride. The number of active threads is reduced by half after each round. The first half of threads in a block is always active (while the second half is inactive). The divergece only happens when the number of active threads in the block in some last rounds smaller than the size of warp ($< 32$).

- In the Kernel 3: the working threads are not changed when compared with the Kernel 2. However, this can help improve the global memory load/store patterns (as explained in "Professional CUDA C Programming" textbook).

### GPU activities

![nvprof of part 1](./Figure/nvprof_part_1.png)

#### Comment

- The program uses most of the time to copy data from Host to Device memory. On the other hand, the program does not use much time to copy data back to Host memory from the Device. This is only true in this case since the amount of data in Device-to-Host-copying process is really small. So I deduce that in general, the copy data processes between Host and Device memory consomue most of the program execution time.

- The time used by each type of kernel is the same as expected: Kernel 1 ran slowest; Kernel 2 ran more quickly; and Kernel 3 is the quickest in execution time.

## Question 1B


In [ ]:
!nvcc -arch=sm_{major}{minor} --ptxas-options=-v {p1Path} -o {p1Exe}
!nvprof {p1Exe} 128

In [ ]:
!nvcc -arch=sm_{major}{minor} --ptxas-options=-v {p1Path} -o {p1Exe}
!nvprof {p1Exe}256

In [ ]:
!nvcc -arch=sm_{major}{minor} --ptxas-options=-v {p1Path} -o {p1Exe}
!nvprof {p1Exe} 512

In [ ]:
!nvcc -arch=sm_{major}{minor} --ptxas-options=-v {p1Path} -o {p1Exe}
!nvprof {p1Exe} 1024

Block size | Grid size | Num blocks / SM | Occupancy (%) | Kernel time (ms)
--- | --- | --- | --- | --- 
1024 | 8193 | 1 | 100 | 2.714 
512 | 16385 | 2 | 100 | 1.992 
256 | 32769 | 4 | 100 | 1.632 
128 | 65537 | 8 | 100 | 1.397 

$\text{Num blocks/SM} = \dfrac{ \text{Maximum threads per SM} }{ \text{Threads per block} }$

$\text{Occupancy} = \dfrac{ \text{Number of active warps per SM} }{ \text{Maximum number of warps per SM} }$

Although all kernels have the same ocupancy, the kernel time is decreasing as the block size decreases. Since when the block size decreases, there are more active blocks and warps to provide for each SM. There are also less arithmetic operations in each block for executing, so the execution time for each block decreases. The CPU can reduce the remained sum from each blocks quickly so the final execution time is the less for kernel use the less size of block.

## Question 2A

| Kernel | Execute Time (ms) |
| -------- | -------- |
| Warmup | 0.074 |
| Host | 9522.951 |
| Basic (No shared memory) | 7.795 |
| Using shared memory | 3.977 ||

![](./Figure/nvprof_part_2.png)

Now even the kernel 1 is slower than the transfer data between Host/Device operation. The Device-to-Host-copying process is same as Host-to-Device-copying process in execution time - this proves the point that I have assumed above in Part 1 since in this case, the output result transfered from Device to Host is quite large. 

### Reasoning

It is clear that when we use shared memory, the execution time decreases dramatically. Because in each block, there are many threads that use the same rows/columns of A, B matrix for computing. It is a waste to always get them from the global memory. So for each block, we detect the elements from A and B that we need to have for computing of that block (in C) and copy required elements from A/B to shared memory in block for quicker access speed.

## Question 2B

In [ ]:
from numba import cuda

major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

wkDir = './'
p2Path = f'{wkDir}HW2_P2.cu'
p2Exe = f'{wkDir}P2.exe'

In [ ]:
!nvcc -arch=sm_{major}{minor} {p2Path} -o {p2Exe}
!nvprof {p2Exe}

**For Basic Matrix Multipication**

1. How many floating operations are being performed in your matrix multiply
kernel? Explain

$2 \times n$. For each thread, we compute $n$ floating multiplication with respective $A$ and $B$ element and use $n$ floating addition to sum the result for that thread.

2. How many global memory reads are being performed by your kernel? Explain

$2 \times n$. For each thread, we read $n$ elements from $A$ and $n$ elements from $B$ to compute result for that thread.

3. How many global memory writes are being performed by your kernel? Explain.

If we use constant memory to keep temporary sum result in the computing process, we only need $1$ global memory write when assign that sum to the suitable position in matrix $C$. If we do not use constant memory, we need $n$ times global memory writes to accumulate result to an element in $C$.

**For Tiled Matrix Multipication**

1. How many floating operations are being performed in your matrix multiply
kernel? Explain

$2 \times n$. Each thread also need $n$ floating multiplication with respective $A$ and $B$ element, and $n$ accumulation operations.

2. How many global memory reads are being performed by your kernel? Explain

- If $n \ mod \ \text{TILE\_STRIDE} = 0$ (there is no threads that positioned outside of valid range of $A$ or $B$): $2 \times \lceil \dfrac{n}{\text{TILE\_STRIDE}} \rceil$. Each thread reads $\lceil \dfrac{n}{\text{TILE\_STRIDE}} \rceil$ from $A$ and $\lceil \dfrac{n}{\text{TILE\_STRIDE}} \rceil$ from $B$ to copy them to shared memory.

- If $n \ mod \ \text{TILE\_STRIDE} \neq 0$ (there is threads that positioned outside of valid range of $A$ or $B$):

    - If thread is in range of $A$ and $B$: same as above case.

    - If thread is outside of range of $A$ and $B$: $2 \times \lceil \dfrac{n}{\text{TILE\_STRIDE}} \rceil - 2$. We minus $2$ because this thread is always positioned at the outside range of both $A$ and $B$. Here we do not read any thing from $A$ and $B$ so we do not conduct global memory reads.

3. How many global memory writes are being performed by your kernel? Explain

Same as the basic matrix multiplication case mentioned above.